# Prediction with LSTM model (v.03)
<p style='text-align: right;'>with selectd4.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [1]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-26 11:29:55.236618


In [2]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.model1 import PredictLSTM1
from lib.model2 import PredictLSTM2
from lib.model3 import PredictLSTM3
from lib.model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from lib.util_pred import print_data
from lib.util_pred import flat_data_with_sum, flat_data, get_frequency
from lib.util_pred import save_model, import_mode
from lib.util_pred import get_random_in_list
from lib.util_pred import print_data_with_sort
from lib.util_pred import get_sorted_n_values
from lib.util_pred import dict_key_count
from lib.util_pred import print_list
from lib.util_pred import print_dict_list
from lib.util_pred import print_title
from lib.activation import ActivationOutput, RecurrentActivation
from datetime import datetime

""" db """
from lib.db_operate import conn_db, close_db
from lib.db_operate import init_metric
from lib.db_operate import select_metric, delete_all_metric
from lib.db_operate import insert_metric

from lib.operate import create_model, create_model_v1, get_predicted, create_model_v2

print(f"imported library. ({datetime.now()})")

imported library. (2025-05-26 11:30:12.351570)


In [3]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd4.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
# last = [[1, 5, 18, 20, 30, 35],
last = [[7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 5

write_to_db = True


finished to set environemnt. (2025-05-26 11:32:39.280761)


In [4]:
# data prepare and set base model
dataset = DataLoader(data_dir=data_dir,
                     training_length=training_length,
                     window_prev=window,
                     mode=mode,
                     from_pos=from_pos
                    )

dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300
                  )
    }
print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

DataLoader.preproc_csv origin = 572, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]
 [1172    7    9 ...   40   42   44]]
rDataLoader.preproc_csv aw_np_proc = 572, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [13 28 34 38 42]
 [ 6  7 11 12 17]
 [ 9 24 40 42 44]]
DataLoader.preproc_csv origin = 572, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]
 [1172    7    9 ...   40   42   44]]
rDataLoader.preproc_csv aw_np_proc = 572, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [13 28 34 38 42]
 [ 6  7 11 12 17]
 [ 9 24 40 42 44]]
DataLoader.preproc_csv origin = 572, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [117

In [22]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_units=[[(45, ActivationOutput.selu.name),   # 0
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 1
             (45, ActivationOutput.selu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 2
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 3
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 4
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             ]]
dense_units = [[(45, ActivationOutput.selu.name),   # 0
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.selu.name),], # 1
               [(45, ActivationOutput.selu.name),   # 2
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.tanh.name),   # 3
               ],
               []                                   # 4
              ]
sel_date = datetime(2025, 5, 24, 20, 35)
sel_date_ts = (sel_date.timestamp() / 10000000)
steps = [45, 45,]
metrics = [["accuracy"]]
dropout = [0, 0]
learning_rate = [0.01, 0.05]
last_lstm_return_sequences = [False]
loss = ["binary_crossentropy", "mse"]
output_dense_activation = [ActivationOutput.elu.name, #0
                           ActivationOutput.selu.name, #1
                           ActivationOutput.sigmoid.name, #2
                           ActivationOutput.tanh.name, #3
                           ActivationOutput.softmax.name, #4
                           ActivationOutput.relu.name, #5
                           ActivationOutput.linear.name #6
                          ]
epochs = [10, #0
          20, #1
          25, #2
          50, #3
          100] #4

rand_seed=[sel_date_ts, datetime.now().timestamp(), 0]


lstm_args = {1: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[0],
                          "dense_units": dense_units[0],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             2: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[1],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             3: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[1]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             4: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[2],
                          "dense_units": dense_units[2],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[2]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             5: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[3],
                          "dense_units": dense_units[3],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             }


models = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-26 11:43:48.618938


In [23]:
import json
import os
from datetime import datetime

def listinlist_2_strinlist(list_in_list):
    """ listinlist_2_strinlist """
    str_in_list = []
    for l in list_in_list:
        str_in_list.append(",".join(l))
    return str_in_list      


def change_matched_info(matched_count, matched_list):
    counts = {}
    cnt = 1
    for c in matched_count:
        counts[cnt] = ",".join([ str(i) for i in c])
        cnt += 1
    cnt = 1
    datas = {}
    for d in matched_list:
        datas[cnt] = d
        cnt += 1
    return counts, datas

def write_json(trial, matched_count, matched_list, append=False):
    suffix = datetime.now().strftime('%y%m%d_%H')
    file_name = f"matched_{suffix}.json"
    matched_dict = {}
    if append:
        if os.path.isfile(file_name):
            with open(file_name, "r") as file:
                matched_dict = json.load(file)
                file.close()
    counts, datas = change_matched_info(matched_count, matched_list)
    if len(counts) > 0 and len(datas) > 0:
        matched_dict[trial] = {}
        matched_dict[trial]["counts"] =  counts
        matched_dict[trial]["datas"] = datas
        print(f'matched_dict = {matched_dict}')
        with open(file_name, "w") as file:
            json.dump(matched_dict, file, indent=4)

In [26]:
# all (SMALL): predict #2
def predict_small(trial, last, models, verbose=True):
    matched_cnts_small = [0 for i in range(MAX_MODEL_CNT)]
    selected_fives_small = [None for i in range(MAX_MODEL_CNT)]
    matched_list_small = [None for i in range(MAX_MODEL_CNT)]
    predicted_all_small = [None for i in range(MAX_MODEL_CNT)]

    for i in range(MAX_MODEL_CNT):
        title = f"history #{i+1}"
        model_num = i
        if models[model_num] is None:
            print(f"model #{model_num+1} is None")
        else:
            models[model_num][0].verb = "None"
            matched_cnts_small[model_num], \
            selected_fives_small[model_num], \
            matched_list_small[model_num], \
            predicted_all_small[model_num] = get_predicted(
                title=title,
                model=models[model_num][0],
                mode=mode2,
                use_pre=False,
                last=last,
                verbose=verbose,
                trial=5
            )
    print(f'predicted time : trial={trial} date={datetime.now()}')
    return matched_cnts_small, matched_list_small


# all (SMALL): print predicted
def print_predict_small(matched_cnts_small, matched_list_small):
    print(f'print time : {datetime.now()}')
    #write_json(trial = 3, matched_count=matched_cnts_small, matched_list=matched_list_small, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts_small)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list_small[i])


In [27]:
single_run = False
if single_run:
    model = create_model_v2(id=1,
                            model_type="lstm4",
                            lstm_info=lstm_args[1],
                            dataset=datasets[1],
                            verbose=True)

In [36]:
# all : create model and training
model_version = 0
with open('active.version', 'r') as fd:
    line = fd.read()
    model_version = int(line)
print(f'previous model_version={model_version}')
print(f'open model version file {datetime.now()}')

print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model_v2(id=i,
                         model_type="lstm4",
                         lstm_info=lstm_args[i+1],
                         dataset=lstm_args[i+1]["dataset"],
                         verbose=False
                        )
    models[i] = (model, lstm_args[i+1])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')
model_version += 1
print(f'check to create model({model_version}). {datetime.now()}')
current = 0
data_cols = []
with open('active.version', 'w') as fd:
    fd.write(f'{model_version}')

previous model_version=1
open model version file 2025-05-26 11:57:02.057289
start to train all models. 2025-05-26 11:57:02.057387
1's training. status=start 2025-05-26 11:57:02.057621
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
lstm unit : 3, (45, 'sigmoid')
lstm unit : 4, (45, 'elu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_10, built=True>
num_epoch=50/<class 'int'>
1's training. status=end 2025-05-26 11:58:00.041954
2's training. status=start 2025-05-26 11:58:00.042060
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_11, built=True>
num_epoch=20/<class 'int'>
2's training. status=end 2025-05-26 11:58:19.442822
3's training. status=start 2025-05-26 11:58:19.442921
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_12, built=True>
num_epoch=50/<class 'int'>
3's training. status=end 2025-05-26 

In [37]:
print(f'check to create model({model_version}). {datetime.now()}')

check to create model(2). 2025-05-26 12:00:21.273251


In [38]:
view_model_info = False
if view_model_info:
    for i in range(MAX_MODEL_CNT):
        print(models[i][0].model.summary())

In [39]:
# all with trial : print predicted
run = False
if run:
    write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

In [40]:
write_to_db = True
if write_to_db:
    conn = conn_db('./db/metrics.db')
    need_db_init = False
    init_metric(conn, drop_table=need_db_init)
else:
    conn = None
round_limit=40
for round in range(round_limit+1):
    matched_cnts_all = []
    matched_list = []
    date = str(datetime.now())
    matched_cnts_small, matched_list_small = predict_small(trial=round, 
                                                           last=last,
                                                           models=models,
                                                           verbose=False
                                                          )
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)
    insert_metric(conn=conn, version=model_version, round=round, date=date, model_datas=matched_list)
if write_to_db:
    close_db(conn)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step
predicted time : trial=0 date=2025-05-26 12:00:45.509701
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=1 date=2025-05-26 12:00:46.173617
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=2 date=2025-05-26 12:00:46.775394
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=3 date=20

In [35]:
trial = 1
matched_cnts_all = []
matched_list = []
for t in range(1, trial+1):
    matched_cnts_small, matched_list_small = predict_small(trial=t, 
                                                           last=last,
                                                           models=models)
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)
current +=1


******************************
	history #1
******************************
start predicting. 2025-05-26 11:56:07.478489
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
wanted_data
1 : [7, 9, 24, 40, 42, 44]
2 : [3, 6, 7, 11, 12, 17]
3 : [3, 13, 28, 34, 38, 42]
4 : [5, 12, 24, 29, 32, 42]
-----
selected_five
	[7, 13, 22, 25, 38, 44]
	[3, 4, 11, 15, 30, 34]
	[8, 16, 29, 37, 38, 45]
	[3, 4, 11, 15, 30, 34]
	[7, 13, 22, 25, 38, 44]
checked cnt = {0: 2, 2: 1, 1: 2}
checked
	([3, 4, 11, 15, 30, 34], [])
	([7, 13, 22, 25, 38, 44], [7, 44])
	([40, 26, 39, 3, 25, 4], [40])
	([8, 16, 29, 37, 38, 45], [])
	([42, 45, 41, 32, 43, 35], [42])
end predicting. 2025-05-26 11:56:07.630273
******************************
	history #2
******************************
start predicting. 2025-05-26 11:56:07.630325
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
wanted_data
1 : [7, 9, 24, 40, 42, 44]
2 : [3, 6, 7, 11, 12, 17]
3 : [3, 13, 28, 34, 38, 42]
4 : [5, 12, 24, 29, 32, 42]
-----
selected_five
	[9, 13, 21, 24, 31, 43]
	[11,

In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [ ]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [ ]:
# [3, 6, 7, 11, 12, 17]

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]

```
sqlite> select * from sqlite_master;
table|metrics|metrics|2|CREATE TABLE metrics (
            id INTEGER PRIMARY KEY,
            version INTEGER,
            round INTEGER,
            model_id INTEGER, 
            trial INTEGER,
            date DATETIME,
            metric TEXT,
            matched TEXT,
            matched_size INTEGER
        )
```